In [1]:
import polars as pl
import polars_ds as pds
import numpy as np

# This notebook illustrates the basic usage of this package

You need to create an environment with this package installed to run this notebook. (usually latest version)

# Num Extensions

In [2]:
size = 10_000
df = pl.DataFrame({
    "f": np.sin(list(range(size)))
    , "time_idx": range(size)
    , "dummy": ["a"] * (size // 2) + ["b"] * (size // 2)
    , "actual": np.round(np.random.random(size=size)).astype(np.int32)
    , "predicted": np.random.random(size=size)
    , "dummy_groups":["a"] * (size//2) + ["b"] * (size//2) 
}).with_columns(
    pds.random(0., 1.).alias("x1")
    , pds.random(0., 1.).alias("x2")
    , pds.random(0., 1.).alias("x3")
    , pds.random(0., 1.).alias("a")
    , pds.random(0., 1.).alias("b")
).with_columns(
    y = pl.col("x1") * 0.15 + pl.col("x2") * 0.3 - pl.col("x3") * 1.5 + pds.random() * 0.0001,
    y2 = pl.col("x1") * 0.13 + pl.col("x2") * 0.45 - pl.col("x3") * 0.1 + pds.random() * 0.0001
)
df.head()

f,time_idx,dummy,actual,predicted,dummy_groups,x1,x2,x3,a,b,y,y2
f64,i64,str,i32,f64,str,f64,f64,f64,f64,f64,f64,f64
0.0,0,"""a""",0,0.095409,"""a""",0.443674,0.356614,0.206802,0.536587,0.368138,-0.136659,0.197483
0.841471,1,"""a""",0,0.613826,"""a""",0.994291,0.572558,0.153125,0.389544,0.324823,0.091277,0.371614
0.909297,2,"""a""",0,0.463168,"""a""",0.16921,0.857024,0.098914,0.67952,0.880558,0.134152,0.397821
0.14112,3,"""a""",0,0.107551,"""a""",0.024009,0.377436,0.47966,0.059801,0.974384,-0.602612,0.125022
-0.756802,4,"""a""",1,0.367916,"""a""",0.498938,0.622296,0.314863,0.998811,0.069195,-0.210755,0.313411


In [3]:
# Column-wise Jaccard Similarity. Result should be 0 as they are distinct
df.select(
    pds.jaccard_col("x1", pl.col("x2"))
)

x1
f64
0.0


In [4]:
# FFT. First is real part, second is complex part
# By default, this behaves the same as np's rfft, which returns a non-redundant 
# compact representation of fft output.
df.select(
    pds.rfft("f")
).head()

f
"array[f64, 2]"
"[1.939505, 0.0]"
"[1.939506, 0.000209]"
"[1.939508, 0.000418]"
"[1.939512, 0.000627]"
"[1.939518, 0.000835]"


In [5]:
# FFT. But return the full length
df.select(
    pds.rfft("f", return_full=True)
).shape

(10000, 1)

In [6]:
# Multiple Convolutions at once
# Modes: `same`, `left` (left-aligned same), `right` (right-aligned same), `valid` or `full`
# Method: `fft`, `direct`
# Currently slower than SciPy but provides parallelism because of Polars
df.select(
    pds.convolve("f", [-1, 0, 0, 0, 1], mode = "full", method = "fft"), # column f with the kernel given here
    pds.convolve("a", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
    pds.convolve("b", [-1, 0, 0, 0, 1], mode = "full", method = "direct"),
).head()

f,a,b
f64,f64,f64
1.3944e-15,-0.536587,-0.368138
-0.841471,-0.389544,-0.324823
-0.909297,-0.67952,-0.880558
-0.14112,-0.059801,-0.974384
0.756802,-0.462224,0.298943


In [7]:
# Linear Regression
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

coeffs
list[f64]
"[-0.481871, -0.34932]"


In [8]:
# Linear Regression, multi-target
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f64],list[f64]
"[-0.481871, -0.34932]","[0.087914, 0.406753]"


In [9]:
# If you want the underlying calculation to be done in f32 instead of f64, you may use the following.
# In some cases, f32 can run faster, especially when input data is in f32.
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = [pl.col("y"), pl.col("y2")],
        add_bias=False
    )
).unnest("coeffs")

target_0,target_1
list[f32],list[f32]
"[-0.481871, -0.34932]","[0.087914, 0.406753]"


In [10]:
pds.config.LIN_REG_EXPR_F64 = True

In [11]:
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f64,f64,f64,f64,f64,f64,f64,f64
"""ln(x1+1)""",0.217035,0.001646,131.834254,0.0,0.213808,0.220262,0.994507,0.994505
"""exp(x2)""",0.176178,0.000669,263.419078,0.0,0.174867,0.177489,0.994507,0.994505
"""sin(x3)""",-1.741186,0.001331,-1308.38992,0.0,-1.743794,-1.738577,0.994507,0.994505
"""__bias__""",-0.11044,0.001497,-73.772692,0.0,-0.113374,-0.107505,0.994507,0.994505


In [12]:
pds.config.LIN_REG_EXPR_F64 = False
df.select(
    pds.lin_reg_report(
        # formulaic input is also available for lstsq related queries, 
        # or you can always use polars expressions, e.g. pl.col('x1') + 1, pl.col('x2').exp(), pl.col('x3').sin()
        "ln(x1+1)", "exp(x2)", "sin(x3)",
        target = "y",
        add_bias = True
    ).alias("report")
).unnest("report")

features,beta,std_err,t,p>|t|,0.025,0.975,r2,adj_r2
str,f32,f32,f32,f32,f32,f32,f32,f32
"""ln(x1+1)""",0.217035,0.001646,131.834305,0.0,0.213808,0.220262,0.994507,0.994505
"""exp(x2)""",0.176179,0.000669,263.419464,0.0,0.174868,0.17749,0.994507,0.994505
"""sin(x3)""",-1.741185,0.001331,-1308.389038,0.0,-1.743793,-1.738576,0.994507,0.994505
"""__bias__""",-0.11044,0.001497,-73.773125,0.0,-0.113375,-0.107506,0.994507,0.994505


In [13]:
pds.config.LIN_REG_EXPR_F64 = True

In [14]:
df.select(
    "dummy",
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    ).over(pl.col("dummy"))
)

dummy,coeffs
str,list[f64]
"""a""","[-0.482173, -0.349658]"
"""a""","[-0.482173, -0.349658]"
"""a""","[-0.482173, -0.349658]"
"""a""","[-0.482173, -0.349658]"
"""a""","[-0.482173, -0.349658]"
…,…
"""b""","[-0.48156, -0.348979]"
"""b""","[-0.48156, -0.348979]"
"""b""","[-0.48156, -0.348979]"


In [15]:
# If you want prediction and residue instead of coefficients
df.select(
    "x1",
    "x2",
    "y",
    pds.lin_reg(
        "x1", pl.col("x2"),
        target = "y",
        add_bias=False, 
        return_pred=True
    ).alias("prediction")
).unnest("prediction").head()

x1,x2,y,pred,resid
f64,f64,f64,f64,f64
0.443674,0.356614,-0.136659,-0.338366,0.201707
0.994291,0.572558,0.091277,-0.679126,0.770403
0.16921,0.857024,0.134152,-0.380913,0.515066
0.024009,0.377436,-0.602612,-0.143415,-0.459197
0.498938,0.622296,-0.210755,-0.457804,0.247049


In [16]:
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.482173, -0.349658]"
"""b""","[-0.48156, -0.348979]"


In [17]:
# Lasso
df.group_by("dummy").agg(
    pds.lin_reg(
        pl.col("x1"), pl.col("x2"),
        target = pl.col("y"),
        l1_reg = 0.1,
        add_bias=False
    )
)

dummy,coeffs
str,list[f64]
"""a""","[-0.312083, -0.178628]"
"""b""","[-0.307225, -0.175513]"


In [18]:
# R2 metric of lasso regressions on each group
df.group_by("dummy").agg(
    pds.query_r2(
        actual = pl.col("y"),
        pred = pds.lin_reg(
            pl.col("x1"), pl.col("x2"),
            target = pl.col("y"),
            l1_reg = 0.1,
            return_pred = True,
            add_bias=False
        ).struct.field("pred")
    ).alias("lasso_r2")
)

dummy,lasso_r2
str,f64
"""a""",-0.550403
"""b""",-0.547596


In [19]:
# Rolling regression
df.select(
    "y",
    "x1",
    "x2",
    pds.rolling_lin_reg(
        "x1", "x2",
        target = "y",
        window_size = 5,
        null_policy = "zero"
    ).alias("result")
).unnest("result")

y,x1,x2,coeffs,pred
f64,f64,f64,list[f64],f64
-0.136659,0.443674,0.356614,null,null
0.091277,0.994291,0.572558,null,null
0.134152,0.16921,0.857024,null,null
-0.602612,0.024009,0.377436,null,null
-0.210755,0.498938,0.622296,"[0.156484, -0.248163]",-0.076355
…,…,…,…,…
-0.297814,0.258718,0.402285,"[0.219466, -0.58243]",-0.177523
-0.432051,0.508813,0.60792,"[0.244401, -0.634709]",-0.261498
-1.330316,0.032038,0.380117,"[0.796059, -1.486164]",-0.539412


In [20]:
# Conditional Entropy, should be 0 because x1 is an ID
df.select(
    pds.query_cond_entropy("y", "x1")
)

y
f64
-0.0


In [21]:
# Only want singular values (principal values?)
df.select(
    pds.singular_values("a", "b", "x1")
)

a
list[f64]
"[29.217307, 28.866389, 28.575499]"


In [22]:
# Singular values + The principal components
df.select(
    pds.pca("a", "b")
).unnest("a")

singular_value,weight_vector
f64,list[f64]
28.990401,"[-0.193942, -0.981013]"
28.64432,"[-0.981013, 0.193942]"


In [23]:
# PC1
df.select(
    pds.principal_components("a", "b", k =1).alias("principal_components")
).unnest("principal_components").head()

pc1
f64
0.125116
0.196126
-0.405295
-0.37715
0.328738


# ML Metrics

In [24]:
df.group_by("dummy_groups").agg(
    pds.query_l2("actual", "predicted").alias("l2"),
    pds.query_log_loss("actual", "predicted").alias("log loss"),
    pds.query_binary_metrics(actual="actual", pred="predicted").alias("combo")
).unnest("combo")


dummy_groups,l2,log loss,precision,recall,f,average_precision,roc_auc
str,f64,f64,f64,f64,f64,f64,f64
"""b""",0.329861,0.992297,0.506349,0.507558,0.506953,0.503973,0.505267
"""a""",0.327294,0.986563,0.508097,0.51037,0.509231,0.508294,0.514328


# Str Extension

In [25]:
size = 100_000
df2 = pl.DataFrame({
    "sen":["Hello, world! I'm going to church."] * size,
    "word":["words", "word"] * (size //2)
})
df2.head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""words"""


In [26]:
# Tokenize
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase()).explode().unique()
)

sen
str
"""world"""
"""to"""
"""hello"""
"""church"""
"""going"""


In [27]:
df2.select(
    pds.str_tokenize(pl.col("sen").str.to_lowercase(), stem=True).explode().unique()
)

sen
str
"""church"""
"""go"""
"""world"""
""""""
"""hello"""


In [28]:
df2.select(
    pds.str_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [29]:
# Damerau-Levenshtein
df2.select(
    pds.str_d_leven("word", pl.lit("world"))
).head()

word
u32
2
1
2
1
2


In [30]:
df2.select( # column "word" vs. the word "world"
    pds.str_leven("word", pl.lit("world"), return_sim = True)
).head()

word
f64
0.6
0.8
0.6
0.8
0.6


In [31]:
df2.filter(
    # This is way faster than computing ditance and then doing a filter
    pds.filter_by_levenshtein(pl.col("word"), pl.lit("world"), 1) # <= 1. 
).head()

sen,word
str,str
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""
"""Hello, world! I'm going to chu…","""word"""


In [32]:
df = pl.DataFrame({
    "word":["apple", "banana", "pineapple", "asasasas", "sasasass"],
    "other_data": [1,2,3,4,5]
})
gibberish = ["asasasa", "sasaaasss", "asdasadadfa"]

In [33]:
df.select(
    # Nearest string
    pds.str_nearest("word", word = "banana")
)

word
str
"""banana"""


In [34]:
df.filter(
    # Filters to words that are similar to any word in vocab
    pds.similar_to_vocab(
        pl.col("word"),
        vocab = gibberish,
        threshold = 0.5,
        metric = "lv", # Levenshtein similarity. Other options: dleven, osa, jw
        strategy = "any" # True if the word is similar to any word in vocab. Other options: "all", "avg"
    )
)

word,other_data
str,i64
"""asasasas""",4
"""sasasass""",5


In [35]:
df.select(
    pds.str_leven("word", pl.lit("asasasa"), return_sim=True).alias("asasasa"),
    pds.str_leven("word", pl.lit("sasaaasss"), return_sim=True).alias("sasaaasss"),
    pds.str_leven("word", pl.lit("asdasadadfa"), return_sim=True).alias("asdasadadfa"),
    pds.str_fuzz("word", pl.lit("apples")).alias("LCS based Fuzz match - apples"),
    pds.str_osa("word", pl.lit("apples"), return_sim=True).alias("Optimal String Alignment - apples"),
    pds.str_jw("word", pl.lit("apples")).alias("Jaro-Winkler - apples"),
)


asasasa,sasaaasss,asdasadadfa,LCS based Fuzz match - apples,Optimal String Alignment - apples,Jaro-Winkler - apples
f64,f64,f64,f64,f64,f64
0.142857,0.111111,0.090909,0.833333,0.833333,0.966667
0.428571,0.333333,0.272727,0.166667,0.0,0.444444
0.111111,0.111111,0.090909,0.555556,0.444444,0.5
0.875,0.666667,0.545455,0.25,0.25,0.527778
0.75,0.777778,0.454545,0.25,0.25,0.527778


# Stats Extension

In [36]:
import numpy as np

df = pl.DataFrame({
    "a": [None, None] + list(np.random.normal(size = 998))
})
df.head()

a
f64
null
null
-0.767269
0.00313
-0.195658


In [37]:
# Genenrate random numbers, respecting null positions in reference column (pl.col("a"))
df.with_columns(
    pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_normal(mean = 0.5, std = 1.0).alias("random_normal")
    ).alias("random_normal_that_respects_null_of_a")
).head()

a,random_normal,random_normal_that_respects_null_of_a
f64,f64,f64
null,0.02952,null
null,1.134819,null
-0.767269,1.371542,0.550586
0.00313,0.63823,1.195501
-0.195658,0.458782,-0.772642


In [38]:
# Genenrate random string
df.with_columns(
    pds.random_str(min_size = 1, max_size = 5).alias("random_str"),
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 1, max_size = 5)
    ).alias("random_str_that_respects_null_of_a")
).head()

a,random_str,random_str_that_respects_null_of_a
f64,str,str
null,"""wpt""",null
null,"""s""",null
-0.767269,"""OUX""","""Uo"""
0.00313,"""kcV""","""TOCvD"""
-0.195658,"""0A""","""xnYhr"""


In [39]:
# Genenrate fixed size random string, while respecting column a's nulls
df.with_columns(
    pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random_str(min_size = 5, max_size = 5)
    ).alias("random_str")
).head()

a,random_str
f64,str
null,null
null,null
-0.767269,"""377kd"""
0.00313,"""73Odx"""
-0.195658,"""52WYg"""


In [40]:
df.with_columns(
    # Sample from a normal distribution, using reference column "a" 's mean and std
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("test1") 
    # Sample from uniform distribution, with low = 0 and high = "a"'s max, and respect the nulls in "a"
    , pl.when(pl.col("a").is_null()).then(None).otherwise(
        pds.random(lower = 0., upper = pl.col("a").max()).alias("test2")
    )
).with_columns(
    # Add a random pertubation to test1
    pds.perturb("test1", epsilon=0.001).alias("test1_perturbed")
).head()

a,test1,literal,test1_perturbed
f64,f64,f64,f64
null,-0.770158,null,-0.770602
null,1.674181,null,1.673707
-0.767269,-0.711246,1.602692,-0.71078
0.00313,0.877491,0.293203,0.877977
-0.195658,0.74019,0.998544,0.740207


In [41]:
# New in v0.3.5
# This way, we don't have a reference column, so we cannot respect nulls, but is more convenient to use.
df.with_columns(
    pds.random().alias("[0, 1)"),
    pds.random_normal(pl.col("a").mean(), pl.col("a").std()).alias("Normal"),
    pds.random_int(0, 10).alias("Int from [0, 10)"),
).head()

a,"[0, 1)",Normal,"Int from [0, 10)"
f64,f64,f64,i32
null,0.604179,-0.101169,4
null,0.29779,0.793103,6
-0.767269,0.523753,1.432596,8
0.00313,0.340589,1.048687,2
-0.195658,0.508313,0.57254,9


In [42]:
# Genenrate 2 random sample, both normally distributed
# Run Welch's t test on them, p value should be big since they have equal mean
# Run a normality test. Again, p value should be big since they are normally distributed 

df.with_columns(
    pds.random_normal(0.5, 1.0).alias("test1"),
    pds.random_normal(0.5, 2.0).alias("test2"),
).select(
    pds.ttest_ind("test1", "test2", equal_var=False).alias("t-test"),
    pds.normal_test("test1").alias("normality_test")
).select(
    pl.col("t-test").struct.field("statistic").alias("t-tests: statistics")
    , pl.col("t-test").struct.field("pvalue").alias("t-tests: pvalue")
    , pl.col("normality_test").struct.field("statistic").alias("normality_test: statistics")
    , pl.col("normality_test").struct.field("pvalue").alias("normality_test: pvalue")
)

t-tests: statistics,t-tests: pvalue,normality_test: statistics,normality_test: pvalue
f64,f64,f64,f64
-0.21819,0.82731,0.899817,0.637686


In [43]:
size = 5_000
df = pl.DataFrame({
    "market_id": range(size),
}).with_columns(
    pl.col("market_id").mod(3),
    var1 = pds.random(),
    var2 = pds.random(),
    category_1 = pds.random_int(0, 5),
    category_2 = pds.random_int(0, 10),
)

df.head(5)

market_id,var1,var2,category_1,category_2
i64,f64,f64,i32,i32
0,0.539665,0.956107,3,6
1,0.319426,0.489367,3,8
2,0.585078,0.531523,0,8
0,0.627588,0.529552,3,7
1,0.638371,0.741195,4,1


In [44]:
# In dataframe statistical tests!
df.select(
    pds.ttest_ind("var1", "var2", equal_var=True).alias("t-test"),
    pds.chi2("category_1", "category_2").alias("chi2-test"),
    pds.f_test("var1", group = "category_1").alias("f-test")
)

t-test,chi2-test,f-test
struct[2],struct[2],struct[2]
"{0.390948,0.695844}","{34.955477,0.518125}","{1.501831,0.198782}"


In [45]:
# Can also be done in group by context
print(
    df.group_by("market_id").agg(
        pds.ttest_ind("var1", "var2", equal_var=False).alias("t-test"),
        pds.chi2("category_1", "category_2").alias("chi2-test"),
        pds.f_test("var1", group = "category_1").alias("f-test")
    )
)

shape: (3, 4)
┌───────────┬──────────────────────┬──────────────────────┬─────────────────────┐
│ market_id ┆ t-test               ┆ chi2-test            ┆ f-test              │
│ ---       ┆ ---                  ┆ ---                  ┆ ---                 │
│ i64       ┆ struct[2]            ┆ struct[2]            ┆ struct[2]           │
╞═══════════╪══════════════════════╪══════════════════════╪═════════════════════╡
│ 0         ┆ {0.521838,0.601818}  ┆ {32.62499,0.629907}  ┆ {0.673291,0.61051}  │
│ 1         ┆ {0.952321,0.341004}  ┆ {44.504977,0.156252} ┆ {0.858875,0.487982} │
│ 2         ┆ {-0.801365,0.422978} ┆ {42.258708,0.218781} ┆ {1.040303,0.384979} │
└───────────┴──────────────────────┴──────────────────────┴─────────────────────┘


In [46]:
# Benford's law
df.select(
    first_digit_cnt = pds.query_first_digit_cnt(pl.col("var1")).explode()
).with_columns(
    # This doesn't follow benford's law because it is random data
    first_digit_distribution = pl.col("first_digit_cnt") / pl.col("first_digit_cnt").sum()
)

first_digit_cnt,first_digit_distribution
u32,f64
560,0.112
564,0.1128
520,0.104
599,0.1198
519,0.1038
533,0.1066
554,0.1108
564,0.1128
587,0.1174


# Nearest Neighbors Related Tasks

These queries can be very slow when data/dimension gets huge, even when processed in parallel.

In [47]:
import polars_ds as pds
size = 2000
df = pl.DataFrame({
    "id": range(size), 
}).with_columns(
    pds.random().alias("var1"),
    pds.random().alias("var2"),
    pds.random().alias("var3"),
    pds.random().alias("r"),
    (pds.random() * 10).alias("rh"),
    pl.col("id").cast(pl.UInt32)
)

In [48]:
# Get neighbor count. The point itself is always considered a neighbor to itself.
df.with_columns(
    pds.query_nb_cnt(
        pl.col("var1"), "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = 0.1, # radius 
        dist = "inf", # L Infinity distance 
        parallel = True 
    ).alias("nb_l_inf_cnt")
).head() 

id,var1,var2,var3,r,rh,nb_l_inf_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.759865,0.00065,0.622723,0.801607,9.023676,8
1,0.198118,0.886434,0.455782,0.504492,3.138422,22
2,0.036138,0.105935,0.261494,0.308848,4.914862,17
3,0.800826,0.046575,0.295647,0.788517,0.813773,7
4,0.82943,0.216105,0.055683,0.283498,4.412655,16


In [49]:
df.with_columns(
    pds.query_nb_cnt(
        "var1", "var2", "var3", # Columns used as the coordinates in n-d space, str | pl.Expr 
        r = pl.col("r"), # radius be an expression too
        dist = "l1", # L 1 distance 
        parallel = True 
    ).alias("nb_l1_r_cnt")
).head()

id,var1,var2,var3,r,rh,nb_l1_r_cnt
u32,f64,f64,f64,f64,f64,u32
0,0.759865,0.00065,0.622723,0.801607,9.023676,543
1,0.198118,0.886434,0.455782,0.504492,3.138422,182
2,0.036138,0.105935,0.261494,0.308848,4.914862,46
3,0.800826,0.046575,0.295647,0.788517,0.813773,560
4,0.82943,0.216105,0.055683,0.283498,4.412655,48


In [50]:
# Get ids of the k nearest neighbors. 
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = "id",  # pl.col("id"), str | pl.Expr
        k = 3, 
        dist = "l2", # squared l2
        parallel = True
    ).alias("best friends")
).head() 

id,var1,var2,var3,r,rh,best friends
u32,f64,f64,f64,f64,f64,list[u32]
0,0.759865,0.00065,0.622723,0.801607,9.023676,"[0, 43, … 1402]"
1,0.198118,0.886434,0.455782,0.504492,3.138422,"[1, 1568, … 1811]"
2,0.036138,0.105935,0.261494,0.308848,4.914862,"[2, 658, … 1970]"
3,0.800826,0.046575,0.295647,0.788517,0.813773,"[3, 1291, … 1664]"
4,0.82943,0.216105,0.055683,0.283498,4.412655,"[4, 417, … 1037]"


In [51]:
# Get all neighbors within radius r, call them best friends
print(

df.select(
    pl.col("id"),
    pds.query_radius_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in 3d space
        index = pl.col("id"),
        r = 0.1, 
        dist = "l2", # actually this is squared l2
        parallel = True
    ).alias("best friends"),
).with_columns( # -1 to remove the point itself
    (pl.col("best friends").list.len() - 1).alias("best friends count")
).head()

)

shape: (5, 3)
┌─────┬───────────────────┬────────────────────┐
│ id  ┆ best friends      ┆ best friends count │
│ --- ┆ ---               ┆ ---                │
│ u32 ┆ list[u32]         ┆ u32                │
╞═════╪═══════════════════╪════════════════════╡
│ 0   ┆ [0, 43, … 1309]   ┆ 4                  │
│ 1   ┆ [1, 1568, … 265]  ┆ 11                 │
│ 2   ┆ [2, 658, … 1578]  ┆ 8                  │
│ 3   ┆ [3, 1291, … 1802] ┆ 5                  │
│ 4   ┆ [4, 417, … 35]    ┆ 10                 │
└─────┴───────────────────┴────────────────────┘


In [52]:
# Get ids of the k nearest neighbors and distances
# The point itself is always considered a neighbor to itself, so k + 1 elements will be returned.
df.with_columns(
    pds.query_knn_ptwise(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        index = pl.col("id"),
        k = 3, 
        dist = "l2", # actually this is squared l2
        parallel = True,
        return_dist = True
    ).alias("best_friends_w_dist")
).unnest("best_friends_w_dist").head()

id,var1,var2,var3,r,rh,idx,dist
u32,f64,f64,f64,f64,f64,list[u32],list[f64]
0,0.759865,0.00065,0.622723,0.801607,9.023676,"[0, 43, … 1402]","[0.0, 0.06804, … 0.092415]"
1,0.198118,0.886434,0.455782,0.504492,3.138422,"[1, 1568, … 1811]","[0.0, 0.041166, … 0.073164]"
2,0.036138,0.105935,0.261494,0.308848,4.914862,"[2, 658, … 1970]","[0.0, 0.043193, … 0.076792]"
3,0.800826,0.046575,0.295647,0.788517,0.813773,"[3, 1291, … 1664]","[0.0, 0.041846, … 0.078438]"
4,0.82943,0.216105,0.055683,0.283498,4.412655,"[4, 417, … 1037]","[0.0, 0.02562, … 0.060861]"


In [53]:
# Filter to only points near the given point
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), pl.col("var3"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5, 0.5],
        r = 0.2,
        dist = "l2" # actually this is squared l2, so this is asking for squared l2 <= 0.2
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
11,0.707257,0.269381,0.306163,0.167248,9.662706
12,0.326548,0.874621,0.360277,0.863122,9.867841
16,0.37988,0.885102,0.390812,0.17811,1.331589
22,0.351878,0.376351,0.866139,0.44334,3.927315
25,0.637806,0.341825,0.874122,0.00083,0.336828


In [54]:
# Haversine distance is available when dimension is 2
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), # Columns used as the coordinates in n-d space
        pt = [0.5, 0.5],
        r = 10, # in km
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
30,0.57826,0.499966,0.53985,0.980339,5.807705
113,0.468908,0.475464,0.449139,0.910683,5.006904
200,0.482622,0.543537,0.719931,0.29741,2.582323
258,0.577855,0.533775,0.587322,0.291092,7.0156
266,0.527302,0.569125,0.366295,0.836803,9.201941


In [55]:
df.filter(
    pds.within_dist_from(
        pl.col("var1"), pl.col("var2"), 
        pt = [0.5, 0.5],
        # radius can also be an existing column in the dataframe.
        r = pl.col("rh"), 
        dist = "h" 
    )
).head()

id,var1,var2,var3,r,rh
u32,f64,f64,f64,f64,f64
113,0.468908,0.475464,0.449139,0.910683,5.006904
266,0.527302,0.569125,0.366295,0.836803,9.201941
420,0.580492,0.46781,0.639045,0.874429,9.903981
428,0.455589,0.524218,0.733707,0.137619,6.589152
542,0.49922,0.476873,0.593337,0.396483,8.499413


In [56]:
friends = df.select(
    pl.col("id").cast(pl.UInt64),
    pds.query_radius_ptwise(
        # Columns used as the coordinates in n-d space
        pl.col("var1"), pl.col("var2"), 
        index=pl.col("id"),
        r = 0.02, 
        dist = "l2",
    ).alias("friends")
).with_columns(
    pl.col("friends").list.len().alias("count")
)
friends.head()

id,friends,count
u64,list[u32],u32
0,"[0, 494]",2
1,"[1, 1328]",2
2,"[2, 658, … 213]",5
3,"[3, 579]",2
4,"[4, 1223, 44]",3


# Using PDS Expressions On pl.Series, NumPy arrays, or pd.Series, etc.

The output by default is always a Polars Series. The user gets to choose whether to turn it into NumPy, Pandas, or other data structures.

In [57]:
import pandas as pd
import numpy as np
from polars_ds.compat import compat as pds2

df = pds.frame(size=100_000).select(
    pds.random(0.0, 1.0).round().alias("actual"),
    pds.random(0.0, 1.0).alias("predicted"),
    pds.random_int(0, 3).alias("0-2"),
    pds.random_int(0, 10).alias("0-9"),
    pds.random_str(min_size=1, max_size=2).alias("s1"),
    pds.random_str(min_size=1, max_size=2).alias("s2"),
)
df.head()

/tmp/ipykernel_19087/3354819425.py:3: UserWarning: The compatibility layer is considered experimental.
  from polars_ds.compat import compat as pds2


actual,predicted,0-2,0-9,s1,s2
f64,f64,i32,i32,str,str
1.0,0.575075,1,4,"""u""","""E4"""
0.0,0.11756,2,0,"""2J""","""3"""
1.0,0.930656,0,3,"""9""","""M"""
1.0,0.23127,2,4,"""W0""","""R"""
1.0,0.268743,2,3,"""Y""","""D"""


In [58]:
df_pd = df.to_pandas()

In [59]:
# Pandas Series
pds2.jaccard_col(df_pd["0-2"], df_pd["0-9"])

jaccard_col
f64
0.3


In [60]:
# Polars Series
print(pds2.query_binary_metrics(df["actual"], df["predicted"]).struct.unnest())
# Pyarrow Series
print(pds2.query_binary_metrics(df["actual"].to_arrow(), df["predicted"].to_arrow()).struct.unnest())
# NumPy
print(pds2.query_binary_metrics(df["actual"].to_numpy(), df["predicted"].to_numpy()).struct.unnest())

shape: (1, 5)
┌───────────┬─────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall  ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---     ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64     ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪═════════╪══════════╪═══════════════════╪══════════╡
│ 0.4973    ┆ 0.49996 ┆ 0.498626 ┆ 0.498115          ┆ 0.497285 │
└───────────┴─────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬─────────┬──────────┬───────────────────┬──────────┐
│ precision ┆ recall  ┆ f        ┆ average_precision ┆ roc_auc  │
│ ---       ┆ ---     ┆ ---      ┆ ---               ┆ ---      │
│ f64       ┆ f64     ┆ f64      ┆ f64               ┆ f64      │
╞═══════════╪═════════╪══════════╪═══════════════════╪══════════╡
│ 0.4973    ┆ 0.49996 ┆ 0.498626 ┆ 0.498115          ┆ 0.497285 │
└───────────┴─────────┴──────────┴───────────────────┴──────────┘
shape: (1, 5)
┌───────────┬─────────┬──────────┬

In [61]:
# NumPy Arrays
pds2.psi(
    np.random.random(size = 1000), 
    np.random.random(size = 1000), 
    n_bins = 5, 
    return_report = True
).struct.unnest()

<=,baseline_pct,actual_pct,psi_bin
f64,f64,f64,f64
0.168922,0.2,0.161,0.00846
0.401733,0.2,0.228,0.003669
0.601501,0.2,0.187,0.000874
0.793278,0.2,0.204,0.000079
inf,0.2,0.22,0.001906


In [62]:
pds2.query_cid_ce(
    np.random.random(size = 1000),
)

cid_ce
f64
13.236203


In [63]:
pds2.query_c3_stats(
    pl.Series(values=np.random.random(size = 1000)), 
    lag = 3
)

c3_stats
f64
0.119764


In [64]:
pds2.str_leven(df["s1"], df["s2"]).head() 

str_leven
u32
2
2
1
2
1
2
1
1
1
